# matching models solved by linear programming

In [66]:
# SETUP
using JuMP
using GLPK
using Random
using LinearAlgebra
include("functions.jl")

solve_model (generic function with 2 methods)

we start with deterministic many-to-many matching models

the solve_model function is defined such that a matching is found to maximize total surplus (under transferable utilities) and satisfy usual constraints

In [74]:
choice = 2
show_s = true # if we want to print out the match surplus matrix
pr_match = true #if we want to print out match pattern

if choice == 1
    S = repeat(reshape(1:8,:,1),1,4) 
    q = [1,2,3,2] #quota for each firm 
    println("8 workers with increasing ability sorted into 4 firms with 8 spots")
    solve_model(S, pr_match)
elseif choice == 2
    S = repeat(reshape(1:12,:,1),1,4) 
    println("12 workers with increasing ability sorted into 4 firms with 8 spots")
    solve_model(S, pr_match)
elseif choice == 3
    S = float(repeat(reshape(1:12,:,1),1,4))
    S[:,1] .+= 0.5 
    S[:,4] .+= 0.6 
    q = [2,3,1,2]
    println("12 workers into 4 firms/8 jobs with different firm productivity")
    solve_model(S, pr_match)
end 

show_s && display(S)

# notice how in case 3 most productive firms choose first from descending order

12×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0

12 workers with increasing ability sorted into 4 firms with 8 spots
Worker values: [0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
Firm values:   [4.0, 4.0, 4.0, 4.0]


12×4 Matrix{Int64}:
  1   1   1   1
  2   2   2   2
  3   3   3   3
  4   4   4   4
  5   5   5   5
  6   6   6   6
  7   7   7   7
  8   8   8   8
  9   9   9   9
 10  10  10  10
 11  11  11  11
 12  12  12  12

now let's look at a model where there's some noise 